# TP1 - Clean : Nettoyage des Données

Ce notebook démontre l'utilisation de l'API FastAPI pour le nettoyage de données.

## Objectifs

1. Générer un dataset avec des défauts (missing, doublons, outliers)
2. Obtenir un rapport qualité
3. Fitter un pipeline de nettoyage
4. Appliquer le nettoyage
5. Comparer avant/après

In [ ]:
import requests
import pandas as pd
import json

# Configuration de l'API
BASE_URL = "http://localhost:8000"

print("✅ Imports réussis")

: 

## Étape 1 : Générer un dataset avec défauts

Utilisons l'endpoint  avec 

In [ ]:
# Générer le dataset
response = requests.post(
    f"{BASE_URL}/dataset/generate",
    json={
        "phase": "clean",
        "seed": 42,
        "n": 1000
    }
)

data = response.json()
dataset_id = data["meta"]["dataset_id"]

print(f"Dataset ID: {dataset_id}")
print(f"Nombre de lignes: {data['result']['n_rows']}")
print(f"Colonnes: {data['result']['columns']}")

# Afficher un échantillon
df_sample = pd.DataFrame(data['result']['data_sample'])
df_sample.head(10)

## Étape 2 : Obtenir un rapport qualité (avant nettoyage)

Utilisons  pour analyser les défauts

In [ ]:
# Obtenir le rapport
response = requests.get(f"{BASE_URL}/clean/report/{dataset_id}")
report_data = response.json()

report = report_data["report"]

print("📊 Rapport Qualité - AVANT nettoyage
")
print(f"Nombre total de lignes: {report['n_rows']}")
print(f"Nombre de doublons: {report['duplicates']}")
print("
Missing values par colonne:")
for col, stats in report['missing_values'].items():
    print(f"  {col}: {stats['count']} ({stats['rate']*100:.1f}%)")

print("
Outliers par colonne:")
for col, stats in report['outliers'].items():
    print(f"  {col}: {stats['count']} ({stats['rate']*100:.1f}%)")

print("
Types de données:")
for col, dtype in report['data_types'].items():
    print(f"  {col}: {dtype}")

## Étape 3 : Fitter un pipeline de nettoyage

Définir les stratégies de nettoyage et apprendre le pipeline

In [ ]:
# Fitter le pipeline
response = requests.post(
    f"{BASE_URL}/clean/fit",
    json={
        "meta": {
            "dataset_id": dataset_id
        },
        "params": {
            "impute_strategy": "mean",
            "outlier_strategy": "clip",
            "categorical_strategy": "one_hot"
        }
    }
)

fit_data = response.json()
cleaner_id = fit_data["result"]["cleaner_id"]

print(f"✅ Pipeline de nettoyage créé: {cleaner_id}")
print("
Règles apprises:")
print(f"  Impute values: {fit_data['report']['rules_learned']['impute_values_count']} colonnes")
print(f"  Outlier bounds: {fit_data['report']['rules_learned']['outlier_bounds_count']} colonnes")
print(f"  Categorical mappings: {fit_data['report']['rules_learned']['categorical_mappings_count']} colonnes")

## Étape 4 : Appliquer le nettoyage

Transformer les données avec le pipeline appris

In [ ]:
# Appliquer la transformation
response = requests.post(
    f"{BASE_URL}/clean/transform",
    json={
        "meta": {
            "dataset_id": dataset_id
        },
        "params": {
            "cleaner_id": cleaner_id
        }
    }
)

transform_data = response.json()
cleaned_dataset_id = transform_data["result"]["processed_dataset_id"]

print(f"✅ Nettoyage appliqué: {cleaned_dataset_id}")
print("
📊 Compteurs de nettoyage:")
counters = transform_data["report"]["counters"]
print(f"  Lignes avant: {counters['rows_before']}")
print(f"  Lignes après: {counters['rows_after']}")
print(f"  Doublons supprimés: {counters['duplicates_removed']}")
print("
  Missing values imputées:")
for col, count in counters['missing_imputed'].items():
    print(f"    {col}: {count}")
print("
  Outliers traités:")
for col, count in counters['outliers_treated'].items():
    print(f"    {col}: {count}")

# Afficher un échantillon nettoyé
df_clean = pd.DataFrame(transform_data['result']['data_sample'])
print("
✅ Échantillon de données nettoyées:")
df_clean.head(10)

## Étape 5 : Comparaison Avant/Après

Visualiser l'amélioration de la qualité des données

In [ ]:
print("📊 Rapport AVANT vs APRÈS
")
print("="*50)

report_before = fit_data['report']['quality_before']
report_after = transform_data['report']['report_after']

print(f"Lignes:  {report_before['n_rows']} → {report_after['n_rows']}")
print(f"Doublons: {report_before['duplicates']} → {report_after['duplicates']}")

print("
Missing values (taux):")
for col in report_before['missing_values'].keys():
    if col in report_after['missing_values']:
        before = report_before['missing_values'][col]['rate'] * 100
        after = report_after['missing_values'][col]['rate'] * 100
        print(f"  {col}: {before:.1f}% → {after:.1f}%")

print("
✅ Nettoyage terminé avec succès !")